In [30]:
import numpy as np
import pandas as pd
import xarray as xr

In [21]:
def get_phase_name(phaseid, ds_phase_names):
    return ds_phase_names['Phase_englisch'][ds_phase_names['Phase_ID'] == str(phaseid)].values[0]

In [29]:
def get_station_locations(dataset, ds_stations):
    ds_stations.index = ds_stations['Stations_id']
    lat = [ds_stations._get_value(row, col) for row, col in zip(dataset['Stations_id'], ['geograph.Breite' for count in range(len(dataset))])] #station_data.lookup(row_labels = dataset['Stations_id'], col_labels = ['geograph.Breite'])
    lon = [ds_stations._get_value(row, col) for row, col in zip(dataset['Stations_id'], ['geograph.Laenge' for count in range(len(dataset))])] #station_data._lookup(dataset['Stations_id'], ['geograph.Laenge'])
    return lat, lon

In [ ]:
def phase_dependent_response(driver_values, t_dev, responses, thresholds):
    #Thresholds are the thresholds in development time where the different growth phases change
    #Responses are the response functions, index starting at 'before the first threshold'
    #driver values are the inputs to the response function
    #t_dev is the (cts) development time
    phase = np.digitize(t_dev, thresholds)
    response = np.zeros(driver_values.shape)
    for phase_index in range(len(responses)):
        response += (phase == phase_index)*responses[phase_index](driver_values) #First brackets indicates if we are at the right phase, second takes the response function for each phase
    return response

In [31]:
def interpolate_xy(x, y, ds):
    #Interpolates the input array onto the (non-gridded e.g. phenology station) coordinates x and y.
    #Note hyras is not stored on the full grid, but on some kind of subset. Not quite sure how this works. Just got to hope the stations are in a hyras gridpoint.
    X_for_interp = xr.DataArray(x, dims="modelpoint")
    Y_for_interp = xr.DataArray(y, dims="modelpoint")
    return ds.interp(x=X_for_interp, y=Y_for_interp)#, kwargs={"fill_value": None})

In [ ]:
def latlon_to_projection(x_coords, y_coords):
    proj_epsg = ccrs.epsg(3034)
    proj_latlon = ccrs.PlateCarree()
    points_epsg = proj_epsg.transform_points(proj_latlon, x_coords, y_coords)
    x_epsg = points_epsg[:, 0]
    y_epsg = points_epsg[:, 1]
    return x_epsg, y_epsg

In [ ]:
def dev_under_response(response, ds_driver, driver_variable, maturity_t_dev):
    # Response is the rate response to driver values. Driver values are the input to this response. Maturity_t_dev is the t_dev value where we should stop running.
    # Put in the date as (day, month, year)
    t_dev = np.zeros(ds_driver[driver_variable].isel(time=0).values.shape) #Continuous development time. When this passes through some thresholds then have change in phase.
    dev_time_series = [t_dev.copy()]
    #i_day = np.datetime64(f'{emergence_date[2]}-{str(emergence_date[1]).zfill(2)}-{str(emergence_date[0]).zfill(2)}')
    i_day = ds_driver['emergence_dates'].values.copy()
    i_day = i_day + np.timedelta64(12, 'h') #so that times are at midday rather than midnight
    time_indexer = xr.DataArray(i_day, dims=["modelpoint"])
    ds_driver.sel(time=time_indexer, method='nearest')
    day_count = 0
    while day_count < 365:#while np.sum(t_dev < maturity_t_dev) > 0.05*len(ds_driver['modelpoint']):
        print(day_count)
        #driver_values = ds_driver.sel(time=np.datetime_as_string(i_day, unit='D')).squeeze().values
        driver_values = ds_driver.sel(time=time_indexer, method='nearest')[driver_variable].values
        #print(t_dev[t_dev<maturity_t_dev])
        print(len(driver_values[t_dev<maturity_t_dev]))
        t_dev += response(driver_values, t_dev) #Rate of change of development stage
        dev_time_series.append(t_dev.copy())
        i_day += np.timedelta64(1,'D')
        day_count += 1
    return np.stack(dev_time_series)

In [ ]:
def get_phase_dates(dev_time_series, thresholds):
    phase_dates_array = np.zeros((len(thresholds), dev_time_series.shape[1]))
    for obs_index in range(dev_time_series.shape[1]):
        phase_dates_array[:, obs_index] = np.digitize(thresholds, dev_time_series[:, obs_index]) #Note that the thresholds are NOT the bins for numpy digitize!
    return phase_dates_array

In [ ]:
def get_modelled_dataset(ds_driver, phase_times, phase_list = []):
    reference_years = np.int64(ds_driver['Referenzjahr'].values)
    Stations_id = np.int64(ds_driver['Stations_id'].values)
    if len(phase_list) == 0:
        computed_phases = [f'modelled time emergence to phase {i + 1}' for i in range(phase_times.shape[0])]
    else:
        computed_phases = [f'modelled time emergence to {phase}' for phase in phase_list]
    data_comparison = {'Stations_id': Stations_id,
                       'Referenzjahr': reference_years,
                       }
    ds_comparison = pd.DataFrame(data_comparison)
    for phase_index, phase in enumerate(computed_phases):
        ds_comparison[phase] = phase_times[phase_index, :]
    ds_comparison.set_index(['Stations_id', 'Referenzjahr'], inplace=True)
    return ds_comparison

In [22]:
class Phenology_set:

    phase_names = pd.read_csv("https://opendata.dwd.de/climate_environment/CDC/help/PH_Beschreibung_Phase.txt", encoding = "latin1", engine='python', sep = r';\s+|;\t+|;\s+\t+|;\t+\s+|;|\s+;|\t+;|\s+\t+;|\t+\s+;')
    station_data = pd.read_csv("https://opendata.dwd.de/climate_environment/CDC/help/PH_Beschreibung_Phaenologie_Stationen_Jahresmelder.txt",sep = ";\s+|;\t+|;\s+\t+|;\t+\s+|;|\s+;|\t+;|\s+\t+;|\t+\s+;", encoding='cp1252', on_bad_lines='skip')

    def __init__(self, address):
        self.phen_data = pd.read_csv(address, encoding = "latin1", engine='python', sep = r';\s+|;\t+|;\s+\t+|;\t+\s+|;|\s+;|\t+;|\s+\t+;|\t+\s+;')
        ## CONVERT DATE TO DATETIME ##
        self.phen_data['Eintrittsdatum'] = pd.to_datetime(self.phen_data['Eintrittsdatum'], format = '%Y%m%d')
        self.phen_data = self.phen_data.where(self.phen_data['Qualitaetsniveau'] == 10).dropna()
        self.T_mean = ''
        self.GDD_driver_data = ''
        self.ordered = False

    ### Functions for sorting out dataset ###
    def drop_columns(self, drop_list):
        for drop_name in drop_list:
            self.phen_data = self.phen_data.drop(drop_name, axis = 1)
    
    def phase_order_name(self, stage_order): #[10, 12, 67, 65, 5, 6, 19, 20, 21, 24, ]
        self.phen_data['Order of phase'] = np.nan
        self.phen_data['Name of phase'] = ''
        for i, phaseid in enumerate(stage_order):
            if len(self.phase_names['Phase_englisch'][self.phase_names['Phase_ID'] == str(phaseid)]) != 0:
                #print(i, phaseid)
                self.phen_data.loc[self.phen_data['Phase_id'] == phaseid, 'Order of phase'] = i
                self.phen_data.loc[self.phen_data['Phase_id'] == phaseid, 'Name of phase'] = get_phase_name(phaseid, self.phase_names)

    def order_phen_dataset(self):
        ## SORT BY TIME ##
        if not(np.isin('Order of phase', self.phen_data.columns)):
            print('Get phase order and names first')
        else:
            self.phen_data.sort_values(by = ['Stations_id', 'Referenzjahr', 'Eintrittsdatum', 'Order of phase'])
            self.ordered = True
    
    def get_time_to_next_stage(self):
        #Note phen_data must be time and station ordered. Only plots time to next stage - naive as doesn't consider missing phases.
        if self.ordered:
            ## CALCULATE TIME TO NEXT STAGE ##
            self.phen_data['Time to next stage'] = self.phen_data['Eintrittsdatum'].shift(-1) - self.phen_data['Eintrittsdatum']
            self.phen_data['Next stage name'] = self.phen_data['Name of phase'].shift(-1)
            ## EXCLUDE CHANGES IN STATION ##
            self.phen_data.loc[self.phen_data['Stations_id'] != self.phen_data['Stations_id'].shift(-1), 'Time to next stage'] = np.nan
            self.phen_data.loc[self.phen_data['Stations_id'] != self.phen_data['Stations_id'].shift(-1), 'Next stage name'] = np.nan
        else:
            print('Order dataset so I can get time to next stage')

    def add_locations(self):
        LAT, LON = get_station_locations(self.phen_data, self.station_data)
        self.phen_data['lat'] = LAT
        self.phen_data['lon'] = LON
        self.phen_data['lat'] = self.phen_data['lat'].map(lambda x: x[0] if isinstance(x, np.float64) == False else x)
        self.phen_data['lon'] = self.phen_data['lon'].map(lambda x: x[0] if isinstance(x, np.float64) == False else x)

    ### Functions for applying GDD model ###
    def get_mean_T(self, T_address):
        self.T_mean = xr.open_dataset(T_address)

    def make_input_array(self):
        just_emergence_phen_data = self.phen_data.where(self.phen_data['Name of phase'] == 'beginning of emergence').dropna()
        just_emergence_phen_data = just_emergence_phen_data.where(just_emergence_phen_data['Eintrittsdatum'] > np.datetime64('2005-01-01')).dropna()
        x_coords = just_emergence_phen_data['lon'].values
        y_coords = just_emergence_phen_data['lat'].values
        #Makes an array to put into GDD model
        print('project to new coords')
        x_epsg, y_epsg = latlon_to_projection(x_coords, y_coords)
        print('interpolate driver to station locations')
        self.GDD_driver_data = interpolate_xy(x_epsg, y_epsg, self.T_mean)
        self.GDD_driver_data['emergence_dates'] = (("modelpoint"), just_emergence_phen_data['Eintrittsdatum'].values)
        self.GDD_driver_data['Stations_id'] = (("modelpoint"), just_emergence_phen_data['Stations_id'].values)
        self.GDD_driver_data['Referenzjahr'] = (("modelpoint"), just_emergence_phen_data['Referenzjahr'].values)
        self.GDD_driver_data = self.GDD_driver_data.assign_coords(modelpoint=np.arange(self.GDD_driver_data.sizes['modelpoint']))

    
    
    

C:\Users\wlwc1989\AppData\Local\Temp\ipykernel_3544\3872937943.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  station_data = pd.read_csv("https://opendata.dwd.de/climate_environment/CDC/help/PH_Beschreibung_Phaenologie_Stationen_Jahresmelder.txt",sep = ";\s+|;\t+|;\s+\t+|;\t+\s+|;|\s+;|\t+;|\s+\t+;|\t+\s+;", encoding='cp1252', on_bad_lines='skip')


In [23]:
Maize_set = Phenology_set("C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\Saved_files\\PH_Jahresmelder_Landwirtschaft_Kulturpflanze_Mais_1936_2023_hist.txt")

In [24]:
Maize_set.drop_columns(['Unnamed: 9'])

In [26]:
Maize_set.phase_order_name([10, 12, 67, 65, 5, 6, 19, 20, 21, 24, ])
Maize_set.order_phen_dataset()

In [28]:
Maize_set.phen_data

,Stations_id,Referenzjahr,Qualitaetsniveau,Objekt_id,Phase_id,Eintrittsdatum,Eintrittsdatum_QB,Jultag,eor,Order of phase,Name of phase
0,7501,1956,10,215,10,1956-05-12,1,132,eor,0.0,beginning of tilling sowing drilling
1,7501,1956,10,215,12,1956-05-26,1,146,eor,1.0,beginning of emergence
2,7501,1957,10,215,10,1957-04-28,1,118,eor,0.0,beginning of tilling sowing drilling
3,7501,1957,10,215,12,1957-05-23,1,143,eor,1.0,beginning of emergence
4,7501,1974,10,215,24,1974-10-28,1,301,eor,9.0,harvest
...,...,...,...,...,...,...,...,...,...,...,...
411725,19914,2022,10,215,10,2022-05-06,1,126,eor,0.0,beginning of tilling sowing drilling
411726,19914,2022,10,215,12,2022-05-16,1,136,eor,1.0,beginning of emergence
411727,19914,2022,10,215,24,2022-08-20,1,232,eor,9.0,harvest
411728,19914,2022,10,215,65,2022-07-09,1,190,eor,3.0,tip of tassel visible


In [19]:
Maize_set.phen_data.columns

Index(['Stations_id', 'Referenzjahr', 'Qualitaetsniveau', 'Objekt_id',
       'Phase_id', 'Eintrittsdatum', 'Eintrittsdatum_QB', 'Jultag', 'eor'],
      dtype='object')